<a href="https://colab.research.google.com/github/DB9653/Projektarbeit/blob/main/FINAL_HASOCOne_HASOC2019_task_b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DRIVE and CUDA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# https://github.com/suman101112/hasoc-fire-2020/blob/main/HASOCOne_team_german_task_b.ipynb
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")



# DATA

In [ ]:
import pandas as pd
import math
from sklearn import preprocessing

In [ ]:
# df_train = pd.read_csv("/content/german_dataset2019.tsv",sep="\t", converters={'tweet_id':int})

df_train = pd.read_csv("/content/german_dataset2019.tsv",sep="\t")
#df_train = df_train.dropna()

df_test = pd.read_csv("/content/hasoc_de_test_gold.tsv",sep='\t')

task = 'task2'
task_2019 = 'task_2'

#2019 datasets also --> da wir uns auf 2019 fokussieren, tauschen wir
# 2019 und 2020

#file_2019_1 = pd.read_csv("2019/german_dataset/german_dataset/hasoc_de_test_gold.tsv",sep='\t')
#file_2019_2 = pd.read_csv("2019/german_dataset/german_dataset/german_dataset.tsv",sep="\t") 

#test
#file_2019_1 = pd.read_csv("/content/hasoc_de_test_gold.tsv",sep='\t', converters={'tweet_id':int})
file_2020_train = pd.read_excel("/content/hasoc_2020_de_train_new_b.xlsx", converters={'tweet_id':int})

#train
#file_2019_2 = pd.read_csv("/content/german_dataset2019.tsv",sep="\t", converters={'tweet_id':int})
file_2020_test = pd.read_csv("/content/german_test_1509.csv", converters={'tweet_id':int})

In [ ]:
#sentences_2019_1 = list(file_2019_1['text'].values)
#sentences_2019_2 = list(file_2019_2['text'].values)


sentences_2020_train = list(file_2020_train['text'].values)
sentences_2020_test = list(file_2020_test['text'].values)

#labels_2019_1 = list(file_2019_1[task_2019].values)
#labels_2019_2 = list(file_2019_2[task_2019].values)

labels_2020_train = list(file_2020_train['task2'].values)
labels_2020_test = list(file_2020_test['task2'].values)

print(f"Len df_train: {len(df_train)}")
display(df_train)

Len df_train: 3819


,text_id,text,task_1,task_2
0,hasoc_de_1,Frank Rennicke – Ich bin stolz https://t.co/Cm...,NOT,NONE
1,hasoc_de_2,ANSEHEN.....und danach bitte TEILEN...TEILEN.....,NOT,NONE
2,hasoc_de_3,#Koeln Mohamed erkennt kein deutsches Recht so...,NOT,NONE
3,hasoc_de_4,#SaudiArabien ist eine brutale islamische Dikt...,NOT,NONE
4,hasoc_de_5,Bundespolizei #München hat im 1. Quartal 2019 ...,NOT,NONE
...,...,...,...,...
3814,hasoc_de_3815,"akquirieren Männer,die sich um die Kinder kümm...",NOT,NONE
3815,hasoc_de_3816,"Ja,schon seit Jahren! Ich muß dran denken,das...",NOT,NONE
3816,hasoc_de_3817,@Kurkamp @wendt_joachim @Schroeder_Live @lamni...,NOT,NONE
3817,hasoc_de_3818,EU Wahlen! AfD für Kindererziehungszeiten für...,NOT,NONE


In [ ]:
labelsss = df_train['task_2']
print(labelsss.value_counts())

NONE    3412
OFFN     210
HATE     111
PRFN      86
Name: task_2, dtype: int64


In [ ]:
total_sentences = list(df_train['text'].values)
total_labels = list(df_train['task_2'].values)

#total_sentences.extend(sentences_2019_1)
#total_sentences.extend(sentences_2019_2)

total_sentences.extend(sentences_2020_train)
total_sentences.extend(sentences_2020_test)

#total_labels.extend(labels_2019_1)
#total_labels.extend(labels_2019_2)

total_labels.extend(labels_2020_train)
total_labels.extend(labels_2020_test)

test_sentences = list(df_test['text'].values)
test_labels = list(df_test['task_2'].values)

# Cleaning, Transforming Encoding

In [ ]:
def clean_text(sentences):
    for index,line in enumerate(sentences):
        if "\n" in line:
            sentences[index] = line.replace("\n","")
    return sentences
        
total_sentences = clean_text(total_sentences)
test_sentences = clean_text(test_sentences)

def clean_labels(labels):
    new_list= []
    for value in labels:
        new_list.append(value.strip())
    return new_list

total_labels = clean_labels(total_labels)
test_labels = clean_labels(test_labels)

le = preprocessing.LabelEncoder()
le.fit(total_labels)
encoded_labels = le.transform(total_labels)
encoded_test_labels = le.transform(test_labels)
print(set(encoded_labels))

print(f"\nLen.: total_sentences: {len(total_sentences)}, encoded labels: {len(encoded_labels)}, test_sentences: {len(test_sentences)}, encoded_test_labels: {len(encoded_test_labels)}")

display(df_test)

{0, 1, 2, 3}

Len.: total_sentences: 6718, encoded labels: 6718, test_sentences: 850, encoded_test_labels: 850


,text_id,text,task_1,task_2
0,hasoc_de_5270,"Die Grüne Intelligenz Bestie 'Eckhardt', Prang...",HOF,HATE
1,hasoc_de_5271,"Das Deutschlandlied, Sinn und Unsinn https:/...",NOT,NONE
2,hasoc_de_5272,#Eurovision https://t.co/24T8fzUI44,NOT,NONE
3,hasoc_de_5273,Unsere ESC Kandidatinnen haben selbst Schuld a...,HOF,OFFN
4,hasoc_de_5274,Scheinbar bekommt Sarah's Conner's Sohn Vincen...,HOF,OFFN
...,...,...,...,...
845,hasoc_de_6115,#Erdogan ruft seine Mops wegen Untauglichkeit ...,HOF,OFFN
846,hasoc_de_6116,Ich würde der Linken Chefin Kipping die 1.000 ...,HOF,OFFN
847,hasoc_de_6117,Ausländer sind Top &amp; Trumpf Schon aufgefa...,HOF,OFFN
848,hasoc_de_6118,Wie der Regierungsjet die Regierungsflieger wi...,HOF,OFFN


In [ ]:
!pip install transformers

from transformers import BertTokenizer


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

max_length = 0
for sentence in total_sentences:
    #print(sentence)
    length = len(tokenizer.tokenize(sentence))
    if length > max_length:
        max_length  = length
print("max token length is: ",max_length)
# max token length obtained is 50
# bert tokens are limited to 514 bytes.



     |████████████████████████████████| 4.0 MB 6.0 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 596 kB 42.2 MB/s 
     |████████████████████████████████| 6.6 MB 35.0 MB/s 
     |████████████████████████████████| 880 kB 40.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=82cbdc728330e92cac9426709ef0dc121fa92927dac3d1b29fef70630a222be4
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

max token length is:  175


In [ ]:


def encoder_generator(sentences,labels):
    
    sent_index = []
    input_ids = []
    attention_masks =[]

    for index,sent in enumerate(sentences):
        
        sent_index.append(index)
        
        encoded_dict = tokenizer.encode_plus(sent,
                                             add_special_tokens=True,
                                             max_length=max_length,
                                             padding='max_length',
                                             truncation = True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])

        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids,dim=0)
    attention_masks = torch.cat(attention_masks,dim=0)
    labels = torch.tensor(labels)
    sent_index = torch.tensor(sent_index)

    return sent_index,input_ids,attention_masks,labels

sent_index,input_ids,attention_masks,encoded_label_tensors = encoder_generator(total_sentences,encoded_labels)
test_sent_index,test_input_ids,test_attention_masks,encoded_test_label_tensors = encoder_generator(test_sentences,encoded_test_labels)
print('Original: ', total_sentences[0])
print('Token IDs:', input_ids[0])
#print(encoded_label_tensors)
#print(encoded_test_label_tensors)



Original:  Frank Rennicke – Ich bin stolz https://t.co/Cm6TD8w1k1 https://t.co/qynXso07Zn
Token IDs: tensor([  101, 11872, 52712, 71609, 10112,   100, 21023, 16292, 47264, 48275,
        14120,   131,   120,   120,   188,   119, 11170,   120,   140, 10147,
        11211, 95936, 11396, 10874, 10759, 10174, 10759, 14120,   131,   120,
          120,   188,   119, 11170,   120,   185, 13379, 12674, 11669, 10929,
        11305, 13966, 10115,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0

# Split into Train, Validation and Test Data

In [ ]:
from torch.utils.data import TensorDataset,random_split

dataset = TensorDataset(input_ids,attention_masks,encoded_label_tensors)
test_dataset = TensorDataset(test_sent_index,test_input_ids,test_attention_masks,encoded_test_label_tensors)

train_size = int(0.75*len(dataset))

val_size = len(dataset)-train_size

train_dataset,val_dataset = random_split(dataset,[train_size,val_size])

print('train data samples is {}'.format(len(train_dataset)))
print("valid data samples is {}".format(len(val_dataset)))
print("test data samples is {}".format(len(test_dataset)))

train data samples is 5038
valid data samples is 1680
test data samples is 850


In [ ]:


from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

bs=8

train_data_loader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=bs)
valid_data_loader = DataLoader(val_dataset,
                              sampler=SequentialSampler(val_dataset),
                              batch_size=bs)
test_data_loader = DataLoader(test_dataset,
                            sampler=SequentialSampler(test_dataset),
                            batch_size=bs)



# MODEL

In [ ]:


from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',
                                                     num_labels=len(le.classes_),
                                                     output_attentions=False,
                                                     output_hidden_states=False,
                                                     )
model.cuda()



Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),lr=2e-5,eps=1e-8)

from transformers import get_linear_schedule_with_warmup

epochs=10
total_steps = len(train_data_loader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps=0,
                                           num_training_steps=total_steps)

In [ ]:


import numpy as np

def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label



In [ ]:


import random
import numpy as np
import time
from sklearn.metrics import classification_report,accuracy_score,f1_score

total_t0 = time.time()

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)



# Train and evaluate model

In [ ]:


def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]]).cuda()

def predictions_labels(preds,labels):
    pred = np.argmax(preds,axis=1).flatten()
    label = labels.flatten()
    return pred,label



In [ ]:


def train():
    total_train_loss = 0
    total_train_acc = 0
    
    model.train() # set model in train mode for batchnorm and dropout layers in bert model
    
    for step,batch in enumerate(train_data_loader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()
            

        """
        loss,logits = model(b_input_ids,
                            attention_mask=b_input_mask,
                            labels=b_labels.long(),
                            return_dict = False)

        """
        
        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = output[0]
        logits = output[1]
            
        total_train_loss+=loss.item()
        total_train_acc+=categorical_accuracy(logits,b_labels).item()
            
        loss.backward()
            
        torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
            
        optimizer.step()
            
        scheduler.step() #go ahead and update the learning rate
            
    avg_train_loss = total_train_loss/len(train_data_loader)
    avg_train_acc = total_train_acc/len(train_data_loader)
    
    return avg_train_loss,avg_train_acc



In [ ]:


def evaluate():
    model.eval()
        
    total_eval_accuracy = 0
    total_eval_loss = 0
    number_of_eval_steps= 0
    
    all_true_labels = []
    all_pred_labels = []

    for batch in valid_data_loader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():

            loss, logits = model(b_input_ids,
                                attention_mask= b_input_mask,
                                labels = b_labels.long(),
                                return_dict = False)
        total_eval_loss+=loss.item()

        logits = logits.detach().cpu().numpy()

        label_ids = b_labels.to('cpu').numpy()

        pred,true = predictions_labels(logits,label_ids)
        
        all_pred_labels.extend(pred)
        all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    macro_f1_score = f1_score(all_pred_labels,all_true_labels,average='macro')
    
    avg_val_loss = total_eval_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))
    
    return avg_val_loss,avg_val_accuracy,macro_f1_score



In [ ]:


import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs



In [ ]:


epochs = 7

best_macro_f1 = float('0')

for epoch in range(epochs):
    
    start_time = time.time()
    train_loss,train_acc = train()
    valid_loss,valid_acc,macro_f1 = evaluate()
    
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if macro_f1 > best_macro_f1:
        best_macro_f1 = macro_f1
        torch.save(model,'HASOCOne_HASOC2019_model_german_task_b.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.97      0.88      0.92      1509
           2       0.00      0.00      0.00         0
           3       0.75      0.61      0.67       171

    accuracy                           0.85      1680
   macro avg       0.43      0.37      0.40      1680
weighted avg       0.95      0.85      0.90      1680

accuracy = 0.85
Epoch: 01 | Epoch Time: 6m 30s
	Train Loss: 0.600 | Train Acc: 83.37%
	 Val. Loss: 0.518 |  Val. Acc: 85.30%
              precision    recall  f1-score   support

           0       0.02      1.00      0.03         1
           1       0.99      0.87      0.92      1558
           2       0.00      0.00      0.00         1
           3       0.59      0.68      0.63       120

    accuracy                           0.85      1680
   macro avg       0.40      0.64      0.40      1680
weighted avg       0.96      0.85      0.90      1680

accur

# Load trained model and evaluate on test data

In [ ]:


del model
import gc
gc.collect()

model = torch.load('HASOCOne_HASOC2019_model_german_task_b.pt')
model = model.to(device)





In [ ]:


def evaluate_test():
    model.eval()
        
    total_eval_accuracy = 0
    total_eval_loss = 0
    number_of_eval_steps= 0
    
    all_true_labels = []
    all_pred_labels = []
    
    all_sentence_id=[]

    for batch in test_data_loader:
        b_sentence_id = batch[0].to(device)
        b_input_ids = batch[1].to(device)
        b_input_mask = batch[2].to(device)
        b_labels = batch[3].to(device)

        sent_ids = b_sentence_id.to('cpu').numpy()
        all_sentence_id.extend(sent_ids)
        
        with torch.no_grad():

            loss, logits = model(b_input_ids,
                                attention_mask= b_input_mask,
                                labels = b_labels.long(),
                                return_dict = False)
            
        total_eval_loss+=loss.item()

        logits = logits.detach().cpu().numpy()

        label_ids = b_labels.to('cpu').numpy()
        

        pred,true = predictions_labels(logits,label_ids)
        
        all_pred_labels.extend(pred)
        
        all_true_labels.extend(true)

    print(classification_report(all_pred_labels,all_true_labels))
    avg_val_accuracy = accuracy_score(all_pred_labels,all_true_labels)
    
    avg_val_loss = total_eval_loss/len(valid_data_loader)

    print("accuracy = {0:.2f}".format(avg_val_accuracy))
    
    return avg_val_loss,avg_val_accuracy,all_sentence_id,all_pred_labels

valid_loss,valid_acc,all_sentence_id,all_pred_labels = evaluate_test()



              precision    recall  f1-score   support

           0       0.10      0.36      0.15        11
           1       0.97      0.85      0.91       812
           2       0.03      0.09      0.04        23
           3       0.06      0.25      0.09         4

    accuracy                           0.82       850
   macro avg       0.29      0.39      0.30       850
weighted avg       0.93      0.82      0.87       850

accuracy = 0.82


In [ ]:
pred_labels = list(le.inverse_transform(all_pred_labels))

In [ ]:
from collections import OrderedDict
dict_index_y_value = {}

for index,y_value in zip(all_sentence_id,pred_labels):
    dict_index_y_value[index] = y_value

od = OrderedDict(sorted(dict_index_y_value.items()))

sorted_y_predicts = []
for k,v in od.items():
    sorted_y_predicts.append(v)

In [ ]:
df_test['index_col'] = df_test.index


In [ ]:


test_tweet_id = list(df_test['text_id'].values)
test_ID = list(df_test['index_col'].values)

test_final = pd.DataFrame(zip(test_tweet_id,sorted_y_predicts,test_ID),columns=['text_id',task,'index_col'])

if task == 'task1':
    test_final.to_csv("HASOCOne_HASOC2019_submission_DE_A.csv",index=False)

if task == 'task2':
    test_final.to_csv("HASOCOne_HASOC2019_submission_DE_B.csv",index=False)



In [ ]:


del model
import gc
gc.collect()



50

In [ ]:
model = torch.load("/content/HASOCOne_HASOC2019_model_german_task_b.pt")

path = "/content/drive/MyDrive/Colab Notebooks/{HASOCOne_HASOC2019_model_german_task_b.pt}" 
torch.save(model.state_dict(), path)


In [ ]:
#https://github.com/suman101112/hasoc-fire-2020/blob/main/HASOCOne_team_german_task_b.ipynb